### main problems to tackel

> Problem of scaleability to more number of objectives, which arises due there not being sufficiently big perito optimum
>> how to tackel it ? ___  by using some other generalized definiation of dominance and being better than other solution

> To ensure sufficient diversity in OBJECTIVE and DECISION VARIABLE spaces

> To be computationaly feasable for large number of objectives and decision varable space

> To ensure search locally and globally


## General sequence
* `Decide Timeline` :: arhitecture 
    * timeline will affect the mutation & crossover types and make them as per it 



* `Generate ppl` :: solution package virus module
  * parameters to generate the ppl on
    * range of each dimension
    * destribution of number of soln in each dimension
  * how to generate a solution
    * range of each dimension
    * relation among different parameters of soln
  * size of ppl 


* `Generate child set` :: solution package 
  * how to generate child and its parameters 
  * how many child for each soln :: maybe diferent for each soln
  * how many child of each parent survive :: on what basis
  * add them to child set
  * refine the child set
  
  
  
* `Generate cossover set` :: solution package

  * how to do crossover
  * how many will be involved in each crossover
  * how many new will be result of each crossover
  * how to choose some from them
  * add them to crossover set
  * refine the crossover set
  
  

* `Calculate the statistics of all the sets` :: archi package
  * baised on the statistics calculate the "trigger limit" for each trigger parameter
  * and also calculate the triggers baised on the statistics
  
  
  
* `Decide which type of Mutation to do baised on if trigger limit is exceeded along with timeline` :: archi package
  * run that type of mutation
  
  
* `Decide which type of Selection to do ` :: archi package
  * run that type of selection
  
  
* `Repeat form generating child for required genertions`

### solutions are represented as

$X$  = ( $X_d$ , $G_d$ , $G_o$ , $F_x$)

* $X_d \hspace{1mm}$ is the representation of this solution in decision variable space
* $G_d \hspace{1mm}$ is the unit gradient in decision variable space which decide the direction in which the center is located
* $\epsilon \hspace{1mm}$ is user defined parameter
* $G_o \hspace{1mm}$ is the respective gradient in objective space when changed by $G_d$
> i.e. $G_o = \frac{F( X_d +\epsilon*G_d ) - F( X_d )}{\epsilon}$ , $F(*)$ is the objective vector function
* $F_x$ is objective function vector of $X$

### How to generate child of a solution

$child(X)$ = $C^X$ = ( $C_d^X$ , $GC_d^X$ , $GC_o^X$ , $F_{C^X}$ ) 

> $C_d^X$ = $X_d$ + $G_d*\epsilon$ + $u *r*\epsilon $
* $u = random\_unit\_vector()$  
  + this random vector must account differently for each child as to ensure we are searching all orthogonal direction in objective space, hence number of such child depends on the dimension of the objective space
* $r =random([0,1])$

> $GC_d^X$ = $ unit\_vec( \lambda * G_d$ +$(1-\lambda)*\frac{ G_d + u*r}{|| G_d+ u*r ||_2})$
* $\lambda$ is the decay parameter $\in (0,1)$
* $||V||_2$ is $L_2$ norm of $V$ 

>  $GC_o^X = \frac{F( C_d^X +\epsilon*GC_d^X ) - F( C_d^X )}{\epsilon}$
* Same as for $X$

> $F_{C^X}$ is function objective vector of $C^X$

### How to do crossover of 2 solutions

`Crossover basically operates on 2 main parameters of a solution other 2 are derived from this 2 main parts `
> 1.  ( $X_d$ , $GX_d$ ) :: ( $Y_d$, $GY_d$ ) = ( $XY_d$, $GXY_d$ ) , represents crossover of $X$ and $Y$
*  each of $XY_d$ and $GXY_d$ are function of $X_d$ , $GX_d$ , $Y_d$ & $GY_d$

> $XY_d$ = $c_{xy}$*$X_d$+$(1-c_{xy})$*$Y_d$

> $GXY_d$ = norm(  $c_{xy}$*$GX_d$+$(1-c_{xy})$*$GY_d$    )

* $c_{xy}$ can either be $1/2$ for all times over all dimensions
* Or depend on X & Y if X has better decision fitness than Y, then $c_{xy}$ > $1/2$ in that dimension of decision space
* any function baised on fitness of each dimention in decision space
    * $c_{xy} = $ $\frac{Fitness(X)}{Fitness(X)+Fitness(Y)}$
    * $c_{xy} = $ $\frac{e^{Fitness(X)}}{e^{Fitness(Y)}+e^{Fitness(X)}}$
    * implemented with 1/2
    

## How to do selection

1. sort the 3 sets of solutions // sort baised on how many objectives does one solution dominates the other
    * ppl
    * child
    * crossover
2. select some proportion from them (this will be baised on the triggers, which will be determined by the timeline & stats of solution
3. add the selected child set & crossover set to ppl set to get new ppl set


In [43]:
print("""e.g

ppl =[3,0,1,2,6]
ppl sorted = [0,1,2,3,6]
seleted = [0,1,2]
child =[36,446,4446,545]
child sorted = [36,446,545,4446]
selected = [36]
crossover = [2,4,56,-1]
crossover sorted = [-1,2,4,56]
selected = [-1,2,4]

ppl_new = [0,1,2,36,-1,2,4]""")

e.g

ppl =[3,0,1,2,6]
ppl sorted = [0,1,2,3,6]
seleted = [0,1,2]
child =[36,446,4446,545]
child sorted = [36,446,545,4446]
selected = [36]
crossover = [2,4,56,-1]
crossover sorted = [-1,2,4,56]
selected = [-1,2,4]

ppl_new = [0,1,2,36,-1,2,4]


In [31]:
import matplotlib.pyplot as plt
import numpy as np

In [242]:
def objective(X):
    n = X.shape[0]
    # n = 2 
    m = 2
    
    F = np.zeros((m,1))
    
    ## objective functions to maximize
    F[0] = X[0]**2 + 10*X[1]**2
    F[1] = 6*X[0]/(3*X[0]+X[1]**2)
    #F[2] = X[0]*X[1]+X[0]/(X[1]+4)
    
    return F

## Generates a random solution in specified range, which is specified by limit

In [243]:
def generate_solution(dims,limits):
    
    x=[]
    g=[]
    for i in range(dims[0]):
        x.append(np.random.uniform(limits['min'][i],limits['max'][i]))
        g.append(np.random.uniform(-1,1))
    
    x = np.reshape(np.array(x),dims)
    g = np.reshape(np.array(g),dims)
    mod = np.sqrt(g.T.dot(g))
    g/=mod
    
    return {"Xd":x,"Gd":g,"Go":None,"F":None}

## Calculates change in each objective function $G_o$  when changed $X_d$ by $\epsilon*G_d$

* requires $X_d$, $G_d$ , $\epsilon$ and $F$ 

In [244]:
# returns G_o, input X = (Xd, Gd, ?, F)

def gradiant(X,epsilon):
    G = (objective(X['Xd']+epsilon*X['Gd']) - X['F'])/epsilon
    return G

In [245]:
x=generate_solution((2,1),limits={"min":[0,0],"max":[2,2]})
x["F"]=objective(x["Xd"])
x["Go"]=gradiant(x,0.01)

In [246]:
x["Xd"]

array([[0.19139053],
       [1.73036746]])

## Below is Child generating function

In [247]:
def childs(x,count=1,epsilon = 0.01 ,lamb = 0.3 ,grad_epsilon = 0.001):
    child = []
    shape = x['Xd'].shape
    
    while count > 0:
        c = {}
        
        u = np.random.uniform(-1,1,shape)                          # uniform random vector
        mod = np.sqrt(u.T.dot(u))                                  # l_2 norm of this vector 
        u/=mod                                                     # converting to unit vector
        r = np.random.rand()                                       # random number from (0,1)
                      
        c['Xd'] =  x['Xd'] + epsilon*x['Gd'] + 2*epsilon*r*u
                    
                      
        u = 2*u*r + x['Gd']
        mod = np.sqrt(u.T.dot(u))                                  # l_2 norm of this vector 
        u/=mod                                                     # converting to unit vector       
        u = lamb*x['Gd'] + (1-lamb)*u
        mod = np.sqrt(u.T.dot(u))                                  # l_2 norm of this vector 
        u/=mod                                                     # converting to unit vector
        
        c['Gd'] = u
                      
        c['F'] = objective(c["Xd"])   
        c['Go'] = gradiant(x,grad_epsilon)
        
        child.append(c)
        count -=1
                      
    return child

In [248]:
c=childs(x,10)

In [261]:
c[9]['F']

array([[29.08946042],
       [ 0.3276587 ]])

In [263]:
# testing the sorting function
s=p_sort(c)

In [264]:
s[5]['F']

array([[30.23686827],
       [ 0.32663577]])

## Below is dominance and sorting function

In [231]:
# k-perito dominance, a dominates b if k of the a's objective are atlest as good as b

def compare(a,b,k=None):
    
    # counts how many times a domainates b in objective function value
    if k is None:
        k = len(a['F'])//2
        
    scount,ecount,ndcount = 0,0,0
    for i in range(len(a['F'])):
        
        if a['F'][i] > b['F'][i]:
            scount+=1
        if a['F'][i] == b['F'][i]:
            ecount+=1
        if a['F'][i] < b['F'][i]:
            ndcount+=1
            
    # counts how good is gradient of each higher and positive the gradient better the gradiend in objective space
    # note the implementation below doesnot account for the sizes of the change which may be important
    gscount,gecount,gndcount = 0,0,0
    for i in range(len(a['Go'])):
        
        if a['Go'][i] > b['Go'][i]:
            gscount+=1
        if a['Go'][i] == b['Go'][i]:
            gecount+=1
        if a['Go'][i] < b['Go'][i]:
            gndcount+=1
    
    ## below we decide how to decide which one is optimal baised on above metrics
    ## below is very simple example which accounts only for objective value
    
    if scount + ecount >= k:
        return 1
    else:
        return -1
    
    
    
# main sorting happens here
# takes input as a list
# returns the list in assending order of better solution later the solution better it is
# uses Tim Sort in O(nlog(n)) worst case

def p_sort(ppl):
    return sorted(ppl, key=functools.cmp_to_key(compare))

In [7]:
class VGA:
    
    def __init__(self,get_ppl,ppl_parameters,fitness,sort_ppl,get_childs,get_child_params,mutate,crossover,to_trigger):
    
            
        """"this set of functions are just involved in population operations which 
         derive some parameters/statistics of population and can generate the population """
        
        # returns the ppl with supplied parameters
        self.get_ppl = get_ppl
        
        # returns child of an individual on baisis of supplied parameters
        self.get_child_ppl = get_child_ppl
        
        # returns child_ppl parameters given ( ppl parametrs , ppl stats , ppl itself )
        self.get_child_params = get_child_params
        
        # calculates and returns the ppl stats in a dictionary
        self.ppl_stat = ppl_stat 
        
        # returns fitness of the ppl 
        self.fitness =  fitness
        
        # decides if to trigger a perticular process of not
        # has types like type = "strong_selection" , "selective crossover" , "selective mutation" ....
        self.to_trigger = to_trigger
        
        # types of crossover
        # return crossoverd ppl set
        self.random_crossover = random_crossover
        self.selective_crossover = selective_crossover
        
        self.sort_ppl = sort_ppl
        
        # types of selection
        # returns the selected ppl for next generation baised on the ( stats of ppl, parameters, crossover set, child set )
        self.strong_select = strong_select
        self.weak_select = weak_select
        
        # types of mutation
        # mutates the population in place (returns nothing)
        self.random_mutate = random_mutate
        self.selective_mutate = selective_mutate
        
    
    def run

# useless codes below

In [12]:
def merge(p1,p2,f):
    l=[]
    n,m = len(p1)-1,len(p2)-1
    
    while n >= 0 and m >= 0:
        if f(p1[n],p2[m]) == 1:
            l.append(p1[n])
            n-=1
        else:
            l.append(p2[m])
            m-=1
            
    while n >=0 :
        l.append(p1[n])
        n-=1
    
    while m >=0 :
        l.append(p2[m])
        m-=1
    
    return l

def merge_perito_sorting(P,f):
    n = len(P)//2
    p1 = merge_perito_sorting(P[:n],f=f)
    p2 = merge_perito_sorting(P[n:],f=f)
    P = merge(p1,p2,f=f)
    return P

'def merge(p1,p2,f):\n    l=[]\n    n,m = len(p1)-1,len(p2)-1\n    \n    while n >= 0 and m >= 0:\n        if f(p1[n],p2[m]) == 1:\n            l.append(p1[n])\n            n-=1\n        else:\n            l.append(p2[m])\n            m-=1\n            \n    while n >=0 :\n        l.append(p1[n])\n        n-=1\n    \n    while m >=0 :\n        l.append(p2[m])\n        m-=1\n    \n    return l\n\ndef merge_perito_sorting(P,f):\n    n = len(P)//2\n    p1 = merge_perito_sorting(P[:n],f=f)\n    p2 = merge_perito_sorting(P[n:],f=f)\n    P = merge(p1,p2,f=f)\n    return P'

In [19]:
# example for sorting
## returns the fitness
import functools

def fitness(x):
     return x

## returns 1 if a is better than b , 0 if same and -1 is worse
def compare(a, b):
    if fitness(a) > fitness(b):
        return -1
    if fitness(a) == fitness(b):
        return 0
    return 1

l=[1,2,3,4,5,0.8,6]

# sorts in assending order of fitness
s=sorted(l, key=functools.cmp_to_key(compare))

In [20]:
s

[6, 5, 4, 3, 2, 1, 0.8]

In [21]:
compare(4,3)

-1